<a href="https://colab.research.google.com/github/thesaqibmumtaz/DS-PT/blob/main/DS_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#using longitude and latitide
import requests

api_key = "API KEY"
latitude = 35.2238
longitude = -80.614773
width = 1500
height = 1500

url = "https://maps.googleapis.com/maps/api/staticmap"
url += "?center=" + str(latitude) + "," + str(longitude)
url += "&size=" + str(width) + "x" + str(height)
url += "&key=" + api_key
response = requests.get(url)
with open("top_footage.png", "wb") as file:
    file.write(response.content)


In [13]:
#using address
import requests

api_key = "API KEY"
address = "11111 Arlington Church Rd, Charlotte, NC 28227, USA"
width = 1500
height = 1500
zoom = 20
# tilt = 45
url = "https://maps.googleapis.com/maps/api/staticmap"
url += "?center=" + str(address)
url += "&size=" + str(width) + "x" + str(height)
url += "&maptype=satellite"
# url += "&tilt=" + str(tilt)
url += "&zoom=" + str(zoom) 
url += "&key=" + api_key
response = requests.get(url)
with open("top_footage.png", "wb") as file:
    file.write(response.content)


In [ ]:
import cv2
import numpy as np
import mapper

image=cv2.imread("top_footage.jpg")
image=cv2.resize(image,(1000,800)) #resizing cuz cv2 doesn't well with large images
orig=image.copy()

gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)  #RGB To Gray conversion
# cv2_imshow("Title",gray) #if you are using jupiter then change this command to cv2.imshow and for colab this command is correct to use.

blurred=cv2.GaussianBlur(gray,(5,5),0)  #(5,5) is kernel size and 0 is sigma value i.e. amount of blur
# cv2_imshow("Blur",blurred)

edged=cv2.Canny(blurred,35,50)  #Threshold
# cv2_imshow("Canny",edged)

contours,hierarchy=cv2.findContours(edged,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)  #apprximation model
contours=sorted(contours,key=cv2.contourArea,reverse=True)

#the loop extracts the boundary contours of the image
for c in contours:
    p=cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,0.02*p,True)

    if len(approx)==4:
        target=approx
        break
approx=mapper.mapp(target) #find endpoints

pts=np.float32([[0,0],[770,0],[770,770],[0,770]])  #map to 800*800 target window_-Changed to 770*770

op=cv2.getPerspectiveTransform(approx,pts)  #get the bird eye view effect
dst=cv2.warpPerspective(orig,op,(770,770))

print(dst.shape)
status = cv2.imwrite('top_footage1.jpg', dst)

print("Image written to file-system : ", status)
cv2.waitKey(0)
cv2.destroyAllWindows()

(770, 770, 3)
Image written to file-system :  True


In [1]:
#for area conversion of pixel to square feet

In [12]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('top_footage1.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Use Otsu's thresholding method to detect edges in the image
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours in the image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the contour with the maximum area
max_area = 0
max_cnt = None
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > max_area:
        max_area = area
        max_cnt = cnt

# Calculate the area of the building in square pixels
area_pixels = max_area

# Set the conversion factor from pixels to feet
conversion_factor = 0.1607973

# Calculate the area of the building in square feet
area_feet = area_pixels * conversion_factor

# Print the area of the building in square feet
print(f'Area of the building: {area_feet:.2f} square feet')


Area of the building: 4985.92 square feet
